In [ ]:
%run "0a_Workspace_setup.ipynb"

from NHM_helpers.NHM_helpers import (
    hrus_by_poi,
    hrus_by_seg,
    subset_stream_network,
    create_poi_group,
)
from NHM_helpers.map_template import *
from NHM_helpers.NHM_Assist_utilities import make_plots_par_vals

from NHM_helpers.NHM_output_visualization import (
    retrieve_hru_output_info,
    create_sum_var_dataarrays,
    create_mean_var_dataarrays,
    create_sum_var_annual_gdf,
    create_sum_var_annual_df,
    create_sum_var_monthly_df,
    create_var_daily_df,
)
from NHM_helpers.output_plots import *

from ast import literal_eval

poi_id_sel = None

crs = 4326

In [ ]:
# Create dataframes for hydrofabric elements
hru_gdf = create_hru_gdf(
    model_dir,
    GIS_format,
    param_filename,
    nhru_params,
    nhru_nmonths_params,
)

seg_gdf = create_segment_gdf(
    model_dir,
    GIS_format,
    param_filename,
)

nwis_gages_aoi = fetch_nwis_gage_info(
    model_dir,
    control_file_name,
    nwis_gage_nobs_min,
    hru_gdf,
)

poi_df = create_poi_df(
    model_dir,
    param_filename,
    control_file_name,
    hru_gdf,
    nwis_gages_aoi,
    gages_file,
)

default_gages_file = create_default_gages_file(
    model_dir,
    nwis_gages_aoi,
    poi_df,
)

gages_df = read_gages_file(
    model_dir,
    poi_df,
    nwis_gages_file,
    gages_file,
)

### Retrieve pywatershed output file information.  

In [ ]:
plot_start_date, plot_end_date, year_list, output_var_list = retrieve_hru_output_info(
    out_dir,
    water_years,
)

# # Check output variable units
# from ast import literal_eval

# for var in output_var_list:
#     with xr.open_dataset(out_dir / f"{var}.nc") as ds:
#         print(f"{var}: {ds[var].units}")
#         del ds

### Select HRU **output variable** (inches) and a **year** to map.
Optional: If a gage is selected, the map will zoom to the gage.

In [ ]:
from ipywidgets import widgets
from ipywidgets import Text, HBox, VBox, Box
from IPython.display import display

##### Varibale selection widget
output_var_sel = output_var_list[
    1
]  # Set a default value so that the notebook will run without selection

style_date = {"description_width": "initial"}

style_var = {"description_width": "initial"}

v = widgets.Dropdown(
    options=output_var_list,
    value=output_var_list[1],
    description="Output variable for map and plot:",
    layout=widgets.Layout(width="35%"),
    style=style_var,
)


def on_change(change):
    global output_var_sel, sel_flag
    if change["type"] == "change" and change["name"] == "value":
        output_var_sel = v.value
        # sel_flag = True


v.observe(on_change)
# display(v)

##### Year selection widget
list_of_years = year_list.copy()
list_of_years.append(
    "mean_annual"
)  # Append the mean annual so that the default will not be a year
sel_year = list_of_years[
    -1
]  # Set a default value so that the notebook will run without selection

yr = widgets.Dropdown(
    options=list_of_years,
    value=list_of_years[-1],
    description="Time step (year) for map:",
    layout=widgets.Layout(width="35%"),
    style=style_var,
)


def on_change(change):
    global sel_year  # Have to set the var as global so that it is carried outside of the fucntion to the notebook
    if change["type"] == "change" and change["name"] == "value":
        sel_year = yr.value


yr.observe(on_change)

# #################################################
v2 = widgets.Combobox(
    # value=poi_df.poi_id.tolist()[0],
    placeholder="(optional) Enter Gage ID here",
    options=poi_df.poi_id.tolist(),
    description="Plot Gage:",
    ensure_option=True,
    disabled=False,
    layout=widgets.Layout(width="35%"),
    style=style_var,
)


def on_change2(change):
    global poi_id_sel, fig
    if change["type"] == "change" and change["name"] == "value":
        poi_id_sel = v2.value


v2.observe(on_change2)

display(VBox([v, yr, v2]))

### Click here and select "Run Selected Cell and All Below" from the "Run" menu in the toolbar.

In [ ]:
# This is for testing only; can comment out in user version
if poi_id_sel is None:
    poi_id_sel = poi_df.poi_id.tolist()[0]

### Read mapping elements

In [ ]:
pfile_lat, pfile_lon, zoom, cluster_zoom = folium_map_elements(
    hru_gdf, poi_df, poi_id_sel
)

In [ ]:
USGSHydroCached_layer, USGStopo_layer, Esri_WorldImagery, OpenTopoMap = (
    folium_map_tiles()
)

In [ ]:
minimap = create_minimap()

### Create Map

In [ ]:
from IPython.display import display
from folium.plugins import MeasureControl

gdf_output_var_annual, value_min, value_max, var_units, var_desc = (
    create_sum_var_annual_gdf(
        out_dir,
        output_var_sel,
        plot_start_date,
        plot_end_date,
        water_years,
        hru_gdf,
        year_list,
    )
)

# poi_id_sel = None
pfile_lat, pfile_lon, zoom, cluster_zoom = folium_map_elements(
    hru_gdf, poi_df, poi_id_sel
)

m3 = folium.Map()
m3 = folium.Map(
    location=[pfile_lat, pfile_lon],
    # width=1000, height=600,
    tiles=USGSHydroCached_layer,
    zoom_start=zoom,
    control_scale=True,
)
USGStopo_layer.add_to(m3)
OpenTopoMap.add_to(m3)
Esri_WorldImagery.add_to(m3)

m3.add_child(minimap)
m3.add_child(MeasureControl())


fig, hru_map, value_min, value_max, scale_bar_txt = create_annual_output_var_map(
    gdf_output_var_annual,
    output_var_sel,
    sel_year,
)
hru_map.add_to(m3)

marker_cluster_label_hru = create_hru_label(hru_gdf, cluster_zoom)
marker_cluster_label_hru.add_to(m3)

seg_map = create_segment_map_show(seg_gdf)
seg_map.add_to(m3)

poi_marker_cluster, poi_marker_cluster_label = create_poi_marker_cluster(
    poi_df, cluster_zoom
)

poi_marker_cluster.add_to(m3)
poi_marker_cluster_label.add_to(m3)

plugins.Fullscreen(position="topleft").add_to(m3)
folium.LayerControl(collapsed=True, position="bottomright", autoZIndex=True).add_to(m3)

con.print(
    f"NHM simulated {sel_year} {output_var_sel}, {var_units}\n[not u not bold] {var_desc}{scale_bar_txt}[/not u not bold]",
    style="u bold black",
)
# con.print(scale_bar_txt)

m3

## Paste the poi_id in the field below

In [ ]:
v3 = widgets.Combobox(
    value=poi_id_sel,
    placeholder="Select Gage ID here",
    options=poi_df.poi_id.tolist(),
    description="Plot Gage:",
    ensure_option=True,
    disabled=False,
)


def on_change3(change):
    global poi_id_sel, fig
    if change["type"] == "change" and change["name"] == "value":
        poi_id_sel = v3.value


v3.observe(on_change3)

display(v3)

#### Click here and select "Run Selected Cell and All Below" from the "Run" menu in the toolbar.

In [ ]:
# poi_id_sel = "14152000"

### Make plot to view output variable for poi basin and contributing HRUs.

In [ ]:
fig = make_plot_var_for_hrus_in_poi_basin(
    out_dir,
    param_filename,
    water_years,
    hru_gdf,
    poi_df,
    output_var_sel,
    var_units,
    poi_id_sel,
    plot_start_date,
    plot_end_date,
    plot_colors,
)

fig.show()

In [ ]:
def oopla():

hru_gdf, hru_poi_dict = create_poi_group(hru_gdf, poi_df, param_filename)

if poi_id_sel:
    fig = plotly.subplots.make_subplots(
        rows=3,
        cols=1,
        shared_xaxes="columns",
        # shared_yaxes = 'columns',
        start_cell="top-left",
        vertical_spacing=0.1,
        y_title="Water flux, cubic-feet per second",
        subplot_titles=[
            "Annual mean",
            "Monthly mean",
            "Daily",
        ],
        specs=[
            [{"type": "scatter"}],
            [{"type": "scatter"}],
            [{"type": "scatter"}],
        ],
    )
    fig.update_layout(
        title_text=f'The NHM basin water budget flux rates for <br> {poi_id_sel}, {poi_df.loc[poi_df.poi_id == poi_id_sel, "poi_name"].values[0]}',  #
        width=900,
        height=700,
        legend=dict(orientation="v", yanchor="top", y=1, xanchor="right", x=10.0),
        # legend_tracegroupgap = 5,
        font=dict(family="Arial", size=14, color="#7f7f7f"),  # font color
        paper_bgcolor="linen",
        plot_bgcolor="white",
    )

    fig.update_layout(
        title_automargin=True,
        title_font_color="black",
        title_font_size=20,
        title_x=0.5,
        title_y=0.945,
        title_xref="container",
        title_xanchor="center",
    )

    # fig.update_xaxes(range = [daily_plot_df.index[0], daily_plot_df.index[-1]])

    fig.update_layout(font_color="black")
    fig.update_layout(
        legend={"title": "NHM output variable"}
    )  # <--- add only this line

    fig.update_xaxes(ticks="inside", tickwidth=2, tickcolor="black", ticklen=10)
    fig.update_yaxes(ticks="inside", tickwidth=2, tickcolor="black", ticklen=10)

    fig.update_xaxes(
        showline=True, linewidth=2, linecolor="black", gridcolor="lightgrey"
    )
    fig.update_yaxes(
        showline=True, linewidth=2, linecolor="black", gridcolor="lightgrey"
    )

    fig.update_traces(hovertemplate=None)

    fig.update_layout(hovermode="x unified")
    fig.update_layout(
        hoverlabel=dict(
            bgcolor="linen",
            font_size=13,
            font_family="Rockwell",
        )
    )

    daily_fig = go.Figure()
    annual_fig = go.Figure()
    monthly_fig = go.Figure()
    ######################################################
    for var in output_var_list:
        color_sel = var_colors_dict[var]
        leg_only = leg_only_dict[var]

        # JLM TODO: keep this block
        # with xr.open_dataarray(out_dir / f"{var}.nc") as da:
        #     # these machinations are to keep downstream things as they were before some refactoring
        #     da = da.to_dataset().rename_dims({"nhm_id": "nhru"})[da.name]
        #     output_var_daily1 = da.sel(time=slice(plot_start_date, plot_end_date))
        #     output_var_monthly1 = output_var_daily1.resample(time="m").mean()
        #     # Water year annual
        #     output_var_annual1 = output_var_daily1.resample(time="A-SEP").mean()

        var_daily, mean_var_monthly, mean_var_annual, var_units, var_desc = (
            create_mean_var_dataarrays(
                out_dir,
                var,
                plot_start_date,
                plot_end_date,
                water_years,
            )
        )

        def create_var_monthly_for_poi_basin_df(
            mean_var_monthly, #change to mean_var_ts
            var,
            hru_gdf,
            hru_poi_dict,
            poi_id_sel,
        ):

            print(var)
            # Create a dataframe of MONTHLY recharge values for each HRU #######################################
            ds_mean_var_ts_basin1 = mean_var_ts.copy() # Change all variables to this pattern in the funct.
            df_output_var_monthly_basin1 = ds_output_var_monthly_basin1.to_dataframe(
                dim_order=["time", "nhru"]
            )
            df_output_var_monthly_basin1.reset_index(inplace=True, drop=False)
            df_output_var_monthly_basin1.rename(
                columns={var: "output_var"}, inplace=True
            )

            # add the HRU area to the dataframe
            hru_area_df = hru_gdf[["hru_area", "nhm_id"]].set_index(
                "nhm_id", drop=True
            )  # Consider a dictionary from the par file and using .map() instead of merge

            df_output_var_monthly_basin1 = df_output_var_monthly_basin1.merge(
                hru_area_df, how="left", right_index=True, left_on="nhm_id"
            )

            # Add output_var volume to the dataframe
            df_output_var_monthly_basin1["vol"] = (
                df_output_var_monthly_basin1["output_var"]
                * (df_output_var_monthly_basin1["hru_area"] * 6272640)
            ) / (12 * 12 * 12)
            df_output_var_monthly_basin1["cfs"] = (
                (
                    df_output_var_monthly_basin1["output_var"]
                    * (df_output_var_monthly_basin1["hru_area"] * 6272640)
                )
                / (12 * 12 * 12)
            ) / 86400

            # Drop unneeded columns
            df_output_var_monthly_basin1.drop(columns=["nhru"], inplace=True)

            # subset to selcted gage one gage
            df_basin1 = df_output_var_monthly_basin1.loc[
                df_output_var_monthly_basin1["nhm_id"].isin(hru_poi_dict[poi_id_sel])
            ]
            df_basin1.set_index(
                ["time", "nhm_id"], inplace=True, drop=True
            )  # resets the index to that new value and type

            # Calculate basin recharge from individual HRU contributions for plotting
            df_basin_plot1 = df_basin1.groupby(level="time").sum()
            df_basin_plot1["output_var"] = (df_basin_plot1["vol"] * 12 * 12 * 12) / (
                df_basin_plot1["hru_area"] * 6272640
            )  # back to inches

            return df_basin_plot1 # This will be generic but return depends on the ds you pass

            #Then just call the same function on the different ds's to get the new plot_df's

        df_basin_plot1_monthly = create_var_monthly_for_poi_basin_df(
            mean_var_monthly,
            var,
            hru_gdf,
            hru_poi_dict,
            poi_id_sel,
        )




        
        # Create a dataframe of Daily recharge values for each HRU
        ds_output_var_daily_basin1 = output_var_daily1.copy()
        df_output_var_daily_basin1 = ds_output_var_daily_basin1.to_dataframe(
                dim_order=["time", "nhru"]
            )
        df_output_var_daily_basin1.reset_index(inplace=True, drop=False)
        df_output_var_daily_basin1.rename(columns={var: "output_var"}, inplace=True)

        # add the HRU area to the dataframe
        df_output_var_daily_basin1 = df_output_var_daily_basin1.merge(
            hru_area_df, how="left", right_index=True, left_on="nhm_id"
        )

        # Add recharge volume to the dataframe
        df_output_var_daily_basin1["vol"] = (
            df_output_var_daily_basin1["output_var"]
            * (df_output_var_daily_basin1["hru_area"] * 6272640)
        ) / (12 * 12 * 12)
        df_output_var_daily_basin1["cfs"] = (
            (
                df_output_var_daily_basin1["output_var"]
                * (df_output_var_daily_basin1["hru_area"] * 6272640)
            )
            / (12 * 12 * 12)
        ) / 86400

        # Drop unneeded columns
        df_output_var_daily_basin1.drop(columns=["nhru"], inplace=True)


        
        # Create a dataframe of ANNUAL recharge values for each HRU +++++++++++++++++++++++++++++++++++++++++++++++
        ds_output_var_annual_basin1 = output_var_annual1.copy()
        df_output_var_annual_basin1 = ds_output_var_annual_basin1.to_dataframe(
            dim_order=["time", "nhru"]
        )
        df_output_var_annual_basin1.reset_index(inplace=True, drop=False)
        df_output_var_annual_basin1.rename(columns={var: "output_var"}, inplace=True)

        df_output_var_annual_basin1 = df_output_var_annual_basin1.merge(
            hru_area_df, how="left", right_index=True, left_on="nhm_id"
        )

        # Add recharge volume to the dataframe
        df_output_var_annual_basin1["vol"] = (
            df_output_var_annual_basin1["output_var"]
            * (df_output_var_annual_basin1["hru_area"] * 6272640)
        ) / (
            12 * 12 * 12
        )  # Cubic-feet
        df_output_var_annual_basin1["cfs"] = (
            (
                df_output_var_annual_basin1["output_var"]
                * (df_output_var_annual_basin1["hru_area"] * 6272640)
            )
            / (12 * 12 * 12)
        ) / 86400  # Cubic-feet/sec

        # Drop unneeded columns
        df_output_var_annual_basin1.drop(columns=["nhru"], inplace=True)

        ###################################################################################################################

        # subset to selcted gage one gage
        df_basin1 = df_output_var_annual_basin1.loc[
            df_output_var_annual_basin1["nhm_id"].isin(hru_poi_dict[poi_id_sel])
        ]
        df_basin1.set_index(
            ["time", "nhm_id"], inplace=True, drop=True
        )  # resets the index to that new value and type

        # Calculate basin recharge from individual HRU contributions for plotting
        df_basin_plot1 = df_basin1.groupby(level="time").sum()
        df_basin_plot1["output_var"] = (df_basin_plot1["vol"] * 12 * 12 * 12) / (
            df_basin_plot1["hru_area"] * 6272640
        )  # back to inches
        # df_basin_plot['cfs'] =

        annual_fig.add_trace(
            go.Scatter(
                x=df_basin_plot1.index,
                y=(df_basin_plot1.cfs).ravel().tolist(),
                # x=year_list,
                # y= (gdf.loc[gdf.nhm_id == hru_id_sel, year_list].values).ravel().tolist(),
                mode="lines",
                name=var,
                visible=leg_only,
                showlegend=True,
                legendgroup=var,
                # marker=dict(color='lightblue'),
                line_shape="vh",
                line=dict(
                    color=color_sel,
                    width=2,
                    # dash='dot'
                ),
            )
        )

        # # subset to selcted gage one gage
        # df_basin1 = df_output_var_monthly_basin1.loc[
        #     df_output_var_monthly_basin1["nhm_id"].isin(hru_poi_dict[poi_id_sel])
        # ]
        # df_basin1.set_index(
        #     ["time", "nhm_id"], inplace=True, drop=True
        # )  # resets the index to that new value and type

        # # Calculate basin recharge from individual HRU contributions for plotting
        # df_basin_plot1 = df_basin1.groupby(level="time").sum()
        # df_basin_plot1["output_var"] = (df_basin_plot1["vol"] * 12 * 12 * 12) / (
        #     df_basin_plot1["hru_area"] * 6272640
        # )  # back to inches

        monthly_fig.add_trace(
            go.Scatter(
                x=df_basin_plot1.index,
                y=(df_basin_plot1.cfs).ravel().tolist(),
                # x=year_list,
                # y= (gdf.loc[gdf.nhm_id == hru_id_sel, year_list].values).ravel().tolist(),
                mode="lines",
                name=var,
                visible=leg_only,
                showlegend=False,
                legendgroup=var,
                # marker=dict(color='lightblue'),
                line_shape="vh",
                line=dict(
                    color=color_sel,
                    width=2,
                    # dash='dot'
                ),
            )
        )

        # subset to selcted gage one gage
        df_basin1 = df_output_var_daily_basin1.loc[
            df_output_var_daily_basin1["nhm_id"].isin(hru_poi_dict[poi_id_sel])
        ]
        df_basin1.set_index(
            ["time", "nhm_id"], inplace=True, drop=True
        )  # resets the index to that new value and type

        # Calculate basin recharge from individual HRU contributions for plotting
        df_basin_plot1 = df_basin1.groupby(level="time").sum()
        df_basin_plot1["output_var"] = (df_basin_plot1["vol"] * 12 * 12 * 12) / (
            df_basin_plot1["hru_area"] * 6272640
        )  # back to inches

        daily_fig.add_trace(
            go.Scatter(
                x=df_basin_plot1.index,
                y=(df_basin_plot1.cfs).ravel().tolist(),
                # x=year_list,
                # y= (gdf.loc[gdf.nhm_id == hru_id_sel, year_list].values).ravel().tolist(),
                mode="lines",
                name=var,
                visible=leg_only,
                showlegend=False,
                legendgroup=var,
                # marker=dict(color='lightblue'),
                line_shape="vh",
                line=dict(
                    color=color_sel,
                    width=2,
                    # dash='dot'
                ),
            )
        )

    for t in annual_fig.data:
        fig.append_trace(t, row=1, col=1)

    for t in monthly_fig.data:
        fig.append_trace(t, row=2, col=1)

    for t in daily_fig.data:
        fig.append_trace(t, row=3, col=1)

    fig.show()

In [ ]:
fig.show()

In [ ]:
fig